## Question 4
---------
Faites une analyse détaillée et exhaustive des patrons existants dans le jeu de données adulte.
Il est possible que le regroupement de valeurs pour certaines caractéristiques donne des résultat intéressant.
Vous devez aussi, s’il y a lieux, discuter de l’aspect éthique concernant l’utilisation des patrons obtenus.

In [48]:
%matplotlib inline
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [77]:
i=0
j=0

data = open('adult.csv') 
reader = csv.reader(data)

dataSet=[] 
for row in reader:            
    dataSet.append(row)

print(len(dataSet))
def fonctionTb(k):
    
    datacompl2= []
    for k in range(k,k+1):
        for i in range(0,1):
    
            for j in range(0,15):
                age=dataSet[0][j]
                colonne=dataSet[k][j]
                donnee= age +'-->'+ colonne
                
                datacompl2.append(donnee)
                
    return datacompl2
dataClean= []
for l in range (len(dataSet)):
    resultat = fonctionTb(l)

    dataClean.append(resultat)

dataClean.pop(0)
data.close()
#print(dataClean)

48843
[['age-->25', 'workclass-->Private', 'fnlwgt-->226802', 'education-->11th', 'educational-num-->7', 'marital-status-->Never-married', 'occupation-->Machine-op-inspct', 'relationship-->Own-child', 'race-->Black', 'gender-->Male', 'capital-gain-->0', 'capital-loss-->0', 'hours-per-week-->40', 'native-country-->United-States', 'income--><=50K'], ['age-->38', 'workclass-->Private', 'fnlwgt-->89814', 'education-->HS-grad', 'educational-num-->9', 'marital-status-->Married-civ-spouse', 'occupation-->Farming-fishing', 'relationship-->Husband', 'race-->White', 'gender-->Male', 'capital-gain-->0', 'capital-loss-->0', 'hours-per-week-->50', 'native-country-->United-States', 'income--><=50K']]


In [75]:
# cette fonction prends une liste C1, la trier, mapper chaque élément de la liste sur frozenset () 
#et renvoyez cette liste de frozensets.
def createC1(dataClean):
    C1 = []
    for transaction in dataClean:
        for item in transaction:
            if not [item] in C1:
                C1.append([item])
                
    #C1.sort()
    return list(map(frozenset, C1))#utilise un ensemble figé pour pouvoir l'utiliser comme clé dans un dictionnaire

In [76]:
C1 = createC1(dataClean)

#print(C1)

In [78]:
D = list(map(set,dataClean))
D

[{'age-->25',
  'capital-gain-->0',
  'capital-loss-->0',
  'education-->11th',
  'educational-num-->7',
  'fnlwgt-->226802',
  'gender-->Male',
  'hours-per-week-->40',
  'income--><=50K',
  'marital-status-->Never-married',
  'native-country-->United-States',
  'occupation-->Machine-op-inspct',
  'race-->Black',
  'relationship-->Own-child',
  'workclass-->Private'},
 {'age-->38',
  'capital-gain-->0',
  'capital-loss-->0',
  'education-->HS-grad',
  'educational-num-->9',
  'fnlwgt-->89814',
  'gender-->Male',
  'hours-per-week-->50',
  'income--><=50K',
  'marital-status-->Married-civ-spouse',
  'native-country-->United-States',
  'occupation-->Farming-fishing',
  'race-->White',
  'relationship-->Husband',
  'workclass-->Private'}]

In [65]:
#Cette fonction prend trois arguments: un ensemble de données, Ck, une liste d'ensembles candidats et un support min, 
#qui est le support minimum qui vous intéresse. Cette fonction est utilisée pour générer L1 à partir de C1. 
#De plus, cette fonction retourne un dictionnaire avec des valeurs de support.
#C1 est un groupe d’éléments candidat de taille 1. Dans l'algorithme Apriori, nous créons C1, puis nous analysons 
#l'ensemble de données pour voir si ces ensembles d'éléments répondent à nos exigences de support minimales. 
#Les ensembles d'éléments qui répondent à nos exigences minimales deviennent L1. 
#L1 est ensuite combiné pour devenir C2 et C2 sera filtré pour devenir L2.
def scanD(D, Ck, minSupport):
    ssCnt = {}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):
                if not can in ssCnt: ssCnt[can]=1
                else: ssCnt[can] += 1
    numItems = float(len(D))
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key]/numItems
        if support >= minSupport:
            retList.insert(0,key)
        supportData[key] = support
    return retList, supportData


In [66]:
L1,suppDat0 = scanD(D,C1,0.5)
L1

[frozenset({'race-->White'}),
 frozenset({'income--><=50K'}),
 frozenset({'native-country-->United-States'}),
 frozenset({'capital-loss-->0'}),
 frozenset({'capital-gain-->0'}),
 frozenset({'gender-->Male'}),
 frozenset({'workclass-->Private'})]

In [67]:
#La fonction aprioriGen () prendra une liste d'itemsets fréquents, Lk, et la taille des itemsets,
#k, pour produire Ck. Par exemple, les itemsets {0}, {1}, {2} et ainsi de suite seront générés {0,1} {0,2} et {1,2}.
def aprioriGen(Lk, k): #Creer Ck
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk): 
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1==L2: #si k-2 premier elements sont egaux
                retList.append(Lk[i] | Lk[j]) #ajouter ensemble union
    return retList

In [68]:
def apriori(dataClean, minSupport = 0.5):
    C1 = createC1(dataClean)
    D = list(map(set, dataClean))
    L1, supportData = scanD(D, C1, minSupport)
    L = [L1]
    k = 2
    while (len(L[k-2]) > 0):
        Ck = aprioriGen(L[k-2], k)
        Lk, supK = scanD(D, Ck, minSupport)#balayer DB pour obtenir Lk
        supportData.update(supK)
        L.append(Lk)
        k += 1
    return L, supportData #L contient des listes de jeux d'éléments fréquents qui ont rencontré un support minimum de 0.5. 
                          #La variable suppData est un dictionnaire avec les valeurs de support de nos itemsets

In [69]:
#La fonction generateRules () prend trois entrées: une liste d'ensembles d'éléments fréquents, 
#un dictionnaire de données de support pour ces ensembles d'éléments et un seuil de confiance minimal. 
#Il va générer une liste de règles avec des valeurs de confiance que nous pourrons trier plus tard.
def generateRules(L, supportData, minConf=0.7):  #supportData est un dictionnaire qui vient de la fonction scanD 
    bigRuleList = []
    for i in range(1, len(L)):#n'obtenez que les ensembles contenant deux objets ou plus
        for freqSet in L[i]:
            H1 = [frozenset([item]) for item in freqSet]
            if (i > 1):
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList  

def calcConf(freqSet, H, supportData, brl, minConf=0.7):
    # H: items individuels dans le jeu d'associations 
    # brl:la grande liste de règles à modifier
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet]/supportData[freqSet-conseq]
        if conf >= minConf:
            print (freqSet-conseq,'-->',conseq,'conf:',conf)
            brl.append((freqSet-conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    m = len(H[0])
    if (len(freqSet) > (m + 1)):
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)
        if (len(Hmp1) > 1):
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)

In [70]:
L,suppData = apriori(dataClean)

In [71]:
rules= generateRules(L,suppData, minConf=0.7)

frozenset({'workclass-->Private'}) --> frozenset({'race-->White'}) conf: 0.8560136848935291
frozenset({'gender-->Male'}) --> frozenset({'race-->White'}) conf: 0.8800918836140889
frozenset({'capital-gain-->0'}) --> frozenset({'race-->White'}) conf: 0.8521882741535921
frozenset({'race-->White'}) --> frozenset({'capital-gain-->0'}) conf: 0.9143240266270772
frozenset({'capital-loss-->0'}) --> frozenset({'race-->White'}) conf: 0.85356529209622
frozenset({'race-->White'}) --> frozenset({'capital-loss-->0'}) conf: 0.9516306690292611
frozenset({'native-country-->United-States'}) --> frozenset({'race-->White'}) conf: 0.8781940135061143
frozenset({'race-->White'}) --> frozenset({'native-country-->United-States'}) conf: 0.9217230975527992
frozenset({'income--><=50K'}) --> frozenset({'race-->White'}) conf: 0.838514331853048
frozenset({'race-->White'}) --> frozenset({'income--><=50K'}) conf: 0.7460131219769168
frozenset({'workclass-->Private'}) --> frozenset({'capital-gain-->0'}) conf: 0.9239072730